In [12]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import spacy
from textblob import TextBlob

# Function to fetch and extract text from a URL
def fetch_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all(['p', 'div'])  # Include <div> tags
    article_text = ' '.join([p.get_text() for p in paragraphs])
    return article_text

# Function to get the stock name from the symbol
def get_stock_name(symbol):
    stock = yf.Ticker(symbol)
    return stock.info['longName']

# Function to filter text relevant to the stock
def filter_relevant_text(article_text, stock_name, stock_symbol):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(article_text)
    sentences = [sent.text for sent in doc.sents if stock_name in sent.text or stock_symbol in sent.text]
    relevant_text = ' '.join(sentences)
    return relevant_text

# Function to analyze sentiment
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    return sentiment_score

# Function to classify sentiment score into categories
def classify_sentiment(score):
    if score <= -0.6:
        return "Very Negative"
    elif score <= -0.2:
        return "Negative"
    elif score < 0.2:
        return "Neutral"
    elif score < 0.6:
        return "Positive"
    else:
        return "Very Positive"

# Main function to tie everything together
def get_stock_sentiment(symbol, url):
    # Fetch article text
    article_text = fetch_article_text(url)
    
    # Get stock name
    stock_name = get_stock_name(symbol)
    
    # Filter relevant text
    relevant_text = filter_relevant_text(article_text, stock_name, symbol)
    
    # Analyze sentiment
    sentiment_score = analyze_sentiment(relevant_text)
    
    # Classify sentiment
    sentiment_class = classify_sentiment(sentiment_score)
    
    return sentiment_score, sentiment_class

# Example usage
if __name__ == "__main__":
    symbol = 'NVDA'
    url = 'https://www.morningstar.co.uk/uk/news/249407/going-into-earnings-is-nvidia-stock-a-buy-or-a-sell.aspx'  # Replace with an actual URL
    sentiment_score, sentiment_class = get_stock_sentiment(symbol, url)
    print(f"The sentiment score for {symbol} is: {sentiment_score}, which is classified as: {sentiment_class}")







The sentiment score for NVDA is: 0.29250000000000004, which is classified as: Positive
